# Tree Clustering Code 

* Code is based on this implementation of decision tree: https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/
* This code builds decision tree using tree distances 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import scipy.stats as sc
#import shap
#import lime
import sklearn 
import warnings
#import xgboost

import multiprocessing as mp
import time
import pickle 
#import interpret
import collections
import math
import random
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import  RandomForestRegressor
from sklearn.svm import SVC, LinearSVC
from sklearn import preprocessing
#from interpret.glassbox import ExplainableBoostingClassifier
#from interpret import show
from sklearn.preprocessing import LabelEncoder
#from interpret.glassbox import ExplainableBoostingRegressor
#from interpret import show
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import AdaBoostRegressor
%matplotlib inline
import os, sys
#import statsmodels.api as sm
sys.path.append(os.path.abspath("../../../"))

pd.set_option('display.max_columns', None)
#import eli5
#from eli5.sklearn import PermutationImportance


In [2]:
test_final_dist = pd.read_csv("test_final_dist.csv")
full_data_test = pd.read_csv("full_data_test.csv")

In [3]:
test_final_dist = test_final_dist[["i1", "i2", "tree_dist"]]
test_final_dist

,i1,i2,tree_dist
0,0,1,-3.643564
1,0,2,-1.554455
2,0,3,10.277228
3,0,4,10.366337
4,0,5,-1.782178
...,...,...,...
71551,266,264,13.198020
71552,267,264,12.495050
71553,266,265,4.445545
71554,267,265,5.683168


In [4]:
full_data_test_no_clust = full_data_test[["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Deck"]]
full_data_test_no_clust

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Deck
0,0,3,0,35,0,0,7,8
1,0,3,0,20,0,0,7,8
2,0,3,0,26,0,0,7,8
3,1,1,1,58,0,0,146,2
4,1,1,1,35,1,0,83,3
...,...,...,...,...,...,...,...,...
263,1,1,1,31,0,2,164,3
264,0,3,1,21,8,2,69,8
265,0,3,1,41,0,2,20,8
266,0,3,1,31,0,0,7,8


In [5]:
X = full_data_test[["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Deck"]]
y = full_data_test[["clust"]]

In [6]:
small_data_test_no_clust = full_data_test_no_clust.head(25)
small_data_test_no_clust

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Deck
0,0,3,0,35,0,0,7,8
1,0,3,0,20,0,0,7,8
2,0,3,0,26,0,0,7,8
3,1,1,1,58,0,0,146,2
4,1,1,1,35,1,0,83,3
5,0,3,0,19,0,0,7,8
6,0,3,0,25,0,0,7,6
7,0,2,0,47,0,0,15,8
8,1,3,0,32,0,0,7,8
9,0,3,0,22,0,0,7,8


In [7]:
small_dataset = small_data_test_no_clust.to_numpy()
small_dataset

array([[  0,   3,   0,  35,   0,   0,   7,   8],
       [  0,   3,   0,  20,   0,   0,   7,   8],
       [  0,   3,   0,  26,   0,   0,   7,   8],
       [  1,   1,   1,  58,   0,   0, 146,   2],
       [  1,   1,   1,  35,   1,   0,  83,   3],
       [  0,   3,   0,  19,   0,   0,   7,   8],
       [  0,   3,   0,  25,   0,   0,   7,   6],
       [  0,   2,   0,  47,   0,   0,  15,   8],
       [  1,   3,   0,  32,   0,   0,   7,   8],
       [  0,   3,   0,  22,   0,   0,   7,   8],
       [  0,   2,   0,  25,   0,   0,  13,   8],
       [  1,   2,   1,   3,   1,   2,  41,   8],
       [  1,   2,   1,  24,   2,   1,  27,   8],
       [  0,   1,   0,  21,   0,   0,  26,   1],
       [  0,   2,   0,  29,   0,   0,  10,   8],
       [  0,   2,   0,  15,   0,   0,   0,   8],
       [  0,   3,   0,  28,   3,   1,  25,   8],
       [  0,   3,   0,   4,   4,   1,  29,   8],
       [  0,   3,   1,  25,   1,   0,   7,   8],
       [  0,   3,   0,  23,   0,   0,   7,   8],
       [  0,   3,   

In [8]:
dataset = [[0, 2.771244718,1.784783929],
    [0, 1.728571309,1.169761413],
    [0, 3.678319846,2.81281357],
    [0, 3.961043357,2.61995032],
    [0, 2.999208922,2.209014212],
    [1, 7.497545867,3.162953546],
    [1, 9.00220326,3.339047188],
    [1, 7.444542326,0.476683375],
    [1, 10.12493903,3.234550982],
    [1, 6.642287351,3.319983761]]

In [9]:
df = pd.DataFrame(dataset, columns = ['class', 'X1', 'X2'])

In [10]:
df

,class,X1,X2
0,0,2.771245,1.784784
1,0,1.728571,1.169761
2,0,3.678320,2.812814
3,0,3.961043,2.619950
4,0,2.999209,2.209014
5,1,7.497546,3.162954
6,1,9.002203,3.339047
7,1,7.444542,0.476683
8,1,10.124939,3.234551
9,1,6.642287,3.319984


In [11]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset, df):
    feature = df.columns[index]
    left, right = list(), list()
    indleft, indright = list(), list()
    left_df, right_df = pd.DataFrame(columns = df.columns), pd.DataFrame(columns = df.columns)
    for i, row in df.iterrows():
        if row[feature] < value:
            indleft.append(i)
            left_df = left_df.append(row) 
        else:
            indright.append(i)
            right_df = right_df.append(row) 
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    #print(indleft)
    return left, right, indleft, indright, left_df, right_df
 
# Calculate the distance index for a split dataset
def distance_index(left, right, indleft, indright):
    mean_left_dist = 0
    mean_right_dist = 0
    for i in range(len(indleft)):
        for j in range(i+1, len(indleft)):
            nodei_df = test_final_dist[test_final_dist['i1'] == indleft[i]]
            nodei_j_df = nodei_df[nodei_df['i2'] == indleft[j]]
            dist =  float(nodei_j_df["tree_dist"])
            mean_left_dist += dist 
    for i in range(len(indright)):
        for j in range(i+1, len(indright)):
            nodei_df = test_final_dist[test_final_dist['i1'] == indright[i]]
            nodei_j_df = nodei_df[nodei_df['i2'] == indright[j]]
            dist =  float(nodei_j_df["tree_dist"])
            mean_right_dist += dist
    if len(left) == 0 and len(right) == 0:
        return 0
    elif len(left) == 0:
        mean_left_dist = 0
        mean_right_dist = mean_right_dist / len(indright)
        return mean_left_dist * len(left) / (len(left) + len(right)) + mean_right_dist * len(right) / (len(left) + len(right))
    elif len(right) == 0:
        mean_left_dist = mean_left_dist / len(indleft)
        mean_right_dist = 0
        return mean_left_dist * len(left) / (len(left) + len(right)) + mean_right_dist * len(right) / (len(left) + len(right))
    else:
        mean_left_dist = mean_left_dist / len(indleft)
        mean_right_dist = mean_right_dist / len(indright)
        return mean_left_dist * len(left) / (len(left) + len(right)) + mean_right_dist * len(right) / (len(left) + len(right))

# Select the best split point for a dataset
def get_split(dataset, df):
    class_values = list(set(row[0] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(1, len(dataset[0])):
        for row in dataset:
            left, right, indleft, indright, left_df, right_df = test_split(index, row[index], dataset, df)
            groups = left, right, left_df, right_df
            dist = distance_index(left, right, indleft, indright)
            if dist < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], dist, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

In [12]:
# Create a terminal node value
def to_terminal(group):
    outcomes = [row[0] for row in group]
    return max(set(outcomes), key=outcomes.count)
 
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
    left, right, left_df, right_df = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        #print('here')
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, left_df)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(right) <= min_size:
        #print('here')
        node['right'] = to_terminal(right)
    else:
        #print('here')
        node['right'] = get_split(right, right_df)
        split(node['right'], max_depth, min_size, depth+1)

In [13]:
# Build a decision tree
def build_tree(train, df, max_depth, min_size):
    root = get_split(train, df)
    split(root, max_depth, min_size, 1)
    return root
 
# Print a decision tree
def print_tree(node, df, depth=0):
    if isinstance(node, dict):
        print('%s[%s < %.3f]' % ((depth*' ', (df.columns[node['index']]), node['value'])))
        print_tree(node['left'], df, depth+1)
        print_tree(node['right'], df, depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

In [16]:
tree = build_tree(dataset, df, 3, 1)
print_tree(tree, df)

[X2 < 2.813]
 [X2 < 2.209]
  [X1 < 1.729]
   [0]
   [0]
  [X1 < 3.961]
   [0]
   [0]
 [X1 < 3.678]
  [1]
  [1]


In [18]:
tree2 = build_tree(small_dataset, small_data_test_no_clust, 2, 1)

In [19]:
print_tree(tree2, small_data_test_no_clust)

[Fare < 26.000]
 [Fare < 12.000]
  [0]
  [0]
 [Age < 24.000]
  [0]
  [1]
